# Just to hide things

In [ ]:
import pandas as pd
import numpy as np

# Further cleaning

In [ ]:
allowed_values = ['Amber', 'Green', 'Red', 'Amber/red', 'Amber/Green']
result['colour_rating_new'] = result['colour_rating_new'].apply(lambda x: x if x in allowed_values else np.nan)

In [ ]:
result = result.rename(columns={'colour_rating_new': 'colour_rating'})

In [ ]:
result['yearly_forecast_new'] = result['yearly_forecast'].combine_first(result['forecast_incl_NGC'])

In [ ]:
result = result.drop(columns=['yearly_forecast', 'forecast_incl_NGC'])

In [ ]:
result = result.rename(columns={'yearly_forecast_new': 'yearly_forecast'})

In [ ]:
result = result.drop(columns=['yearly_budget'])

In [ ]:
result = result.rename(columns={'total_baseline': 'yearly_budget'})
result = result.rename(columns={'TOTAL Baseline Benefits (Łm)': 'total_benefits'})

In [ ]:
column_order = ['project_name','department','colour_rating','description_aims','rating_comment','start_date','end_date','schedule_comment','yearly_budget','yearly_forecast','wlc_baseline_incl_NCG','variance_comment','budget_comment','year','report_category','project_number','total_benefits','benefits_comment']

In [ ]:
result = result[column_order]

In [ ]:
df_macro = pd.read_csv('../data/macro/GDP.csv',encoding='iso-8859-2', delimiter=';')

In [ ]:
df_macro

In [ ]:
df_macro.info()

In [ ]:
# Schritt 1: Identifizieren der relevanten Zeilen
remained_scheduled_df = df_franzi[df_franzi['schedule_comment'].str.contains('remained scheduled', case=False)]

# Schritt 2: Aktuellsten 'end_date'-Wert für jedes 'project_name' ermitteln
latest_end_dates = df_franzi.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'end_date'])

# Schritt 3: Zuweisen des aktuellsten 'end_date' an die relevanten Zeilen
for index, row in remained_scheduled_df.iterrows():
    latest_end_date = latest_end_dates[row['project_name']]
    if pd.isna(row['end_date']) and pd.notna(latest_end_date):
        df_franzi.at[index, 'end_date'] = latest_end_date

# Ergebnis anzeigen
df_franzi


In [ ]:
# Schritt 2: Aktuellsten 'end_date'-Wert für jedes 'project_name' ermitteln
latest_end_dates = df_franzi.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'start_date'])

# Schritt 3: Zuweisen des aktuellsten 'end_date' an die relevanten Zeilen
for index, row in remained_scheduled_df.iterrows():
    latest_end_date = latest_end_dates[row['project_name']]
    if pd.isna(row['start_date']) and pd.notna(latest_end_date):
        df_franzi.at[index, 'start_date'] = latest_end_date

# Ergebnis anzeigen
df_franzi

In [ ]:
    month_mapping = {
        'jan': '01', 'january': '01',
        'feb': '02', 'february': '02',
        'mar': '03', 'march': '03',
        'apr': '04', 'april': '04',
        'may': '05', 'may': '05',
        'jun': '06', 'june': '06',
        'jul': '07', 'july': '07',
        'aug': '08', 'august': '08',
        'sep': '09', 'september': '09',
        'oct': '10', 'october': '10',
        'nov': '11', 'november': '11',
        'dec': '12', 'december': '12'
    }



In [ ]:
def extract_date_after_to_or_on(comment):
    REGEX = "(to|is|on)\s*(\d{1,2})\s+(\w+)\s+(\d{2,4})"
    x = re.compile(REGEX)
    
    match_REGEX = x.search(comment)      
    
    if match_REGEX:
        day = match_REGEX.group(2)
        month = match_REGEX.group(3)
        year = match_REGEX.group(4)
        if len(day) == 1:
            day = '0'+ day
        month = month_mapping.get(month.lower())
        if len(year) == 2:
            year = '20' + year
        return year +'-'+ month +'-'+ day
    
for index, row in df_franzi.iterrows():
    if pd.isna(row['end_date']):
        extracted_date = extract_date_after_to_or_on(row['schedule_comment'])
        if extracted_date:
            df_franzi.at[index, 'end_date'] = extracted_date

df_franzi



In [ ]:
# Der Code zum Filtern von Zeilen, in denen 'start_date' oder 'end_date' 'NaN' ist
# und die 'schedule_comment' nicht mit 'Exempt' oder '\r\nExempt' beginnen

nan_dates_df = df_franzi[
    ((df_franzi['end_date'].isna())) & 
    (~df_franzi['schedule_comment'].str.match(r'^(Exempt|\r\nExempt)'))
][['start_date', 'end_date', 'schedule_comment']]

# Ausgabe des gefilterten DataFrames
nan_dates_df

In [ ]:
# Der Code zum Filtern von Zeilen, in denen 'start_date' oder 'end_date' 'NaN' ist
# und die 'schedule_comment' nicht mit 'Exempt' oder '\r\nExempt' beginnen

nan_dates_df_start = df_franzi[
    ((df_franzi['start_date'].isna())) & 
    (~df_franzi['schedule_comment'].str.match(r'^(Exempt|\r\nExempt)'))
][['start_date', 'schedule_comment']]

# Ausgabe des gefilterten DataFrames
nan_dates_df_start

In [ ]:
df1 = pd.read_pickle("../data/pickle/cleaned_2021_2023.pkl")
df2 = pd.read_pickle('../data/pickle/cleaned_2018_2020.pkl')
df3 = pd.read_pickle('../data/pickle/cleaned_2014-2017.pkl')
df4 = pd.read_csv('../data/macro/GDP.csv',encoding='iso-8859-2', delimiter=';')

In [ ]:
df = pd.concat([df1, df2, df3], axis=0, ignore_index=True)

In [ ]:
df = df.merge(df4, on='year', how='left')

In [ ]:
df.drop(columns='Unnamed: 0')

In [ ]:
df["colour_rating"] = df["colour_rating"].str.lower()

# Drop rows where 'project_name' contains 'Unnamed:' followed by other characters
df = df[~df['project_name'].str.contains(r'Unnamed:.*', na=False)]

# Optionally, reset the index of the cleaned DataFrame
df = df.reset_index(drop=True)

## Imputing

In [ ]:
latest_start_dates = df.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'start_date'])

In [ ]:
for index, row in df.iterrows():
    latest_start_date = latest_start_dates[row['project_name']]
    if pd.isna(row['start_date']) and pd.notna(latest_start_date):
        df.at[index, 'start_date'] = latest_start_date

In [ ]:
latest_benefits = df.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'total_benefits'])

for index, row in df.iterrows():
    latest_benefit = latest_benefits[row['project_name']]
    if pd.isna(row['total_benefits']) and pd.notna(latest_benefit):
        df.at[index, 'total_benefits'] = latest_benefit

In [ ]:
latest_categories = df.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'report_category'])

for index, row in df.iterrows():
    latest_category = latest_categories[row['project_name']]
    if pd.isna(row['report_category']) and pd.notna(latest_category):
        df.at[index, 'report_category'] = latest_category

In [ ]:
latest_numbers = df.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'project_number'])

for index, row in df.iterrows():
    latest_number = latest_numbers[row['project_name']]
    if pd.isna(row['project_number']) and pd.notna(latest_number):
        df.at[index, 'project_number'] = latest_number

In [ ]:
df['description_aims'].unique().tolist()

In [ ]:
df_with_dummies = pd.get_dummies(df, columns=['department', 'colour_rating'], drop_first=True)


In [ ]:
df["start_date"] = pd.to_datetime(df["start_date"])
df["end_date"] = pd.to_datetime(df["end_date"])
# Calculate the duration in years
df["year_duration"] = (df["end_date"] - df["start_date"]).dt.days / 365.25

# Optionally, round the duration to a specific number of decimal places
df["year_duration"] = df["year_duration"].round(2)

In [ ]:
df['start_year'] = df['start_date'].dt.year
df['end_year'] = df['end_date'].dt.year

## Add macro data

In [ ]:
macro_columns = pd.read_csv('../data/macro/GDP.csv', encoding='iso-8859-2', delimiter=';')

In [ ]:
df = df.drop(columns=['GDP', 'annual_earning_ft', 'unemployment_rate', 'youth_unemployment_rate', 'inflation_rate', 'population', 'gov_debt', 'tax_revenue', 'revenue_excl_grants', 'grants_and_other_revenue'])

In [ ]:
df = df.merge(macro_columns, on='year', how='left')

## Final Pickle

In [ ]:
df = pd.read_pickle('../data/pickle/final_pickle.pkl')

## Preprocessing

In [ ]:
df = df.drop(columns=['description_aims', 'rating_comment', 'start_date',
       'end_date', 'schedule_comment', 'yearly_budget', 'yearly_forecast', 'variance_comment', 'budget_comment',
       'report_category', 'project_number', 'total_benefits',
       'benefits_comment', 'population', 'revenue_excl_grants', 'grants_and_other_revenue'])

In [ ]:
df['year'].unique()

In [ ]:
df['year'] = df['year'].astype(int)

In [ ]:
df = df.dropna(subset=['year'])

In [ ]:
df = pd.read_pickle('../data/pickle/final_wo_errors.pkl')

# Start from here

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_pickle('../data/pickle/excluded_outliers.pkl')

### Grouping

In [3]:
colour_rating_columns = [col for col in df.columns if col.startswith('colour_rating_')]
department_columns = [col for col in df.columns if col.startswith('department_')]

def latest_value(df, column):
    try:
        return df.loc[df['year'].idxmax(), column]
    except Exception as e:
        print(f"Error in Group {df.name} for column {column}: {e}")
        return None

agg_dict = {
    'wlc_baseline_incl_NCG': 'max',
    'year_duration': 'max',
    'end_year': 'max',
    'start_year': 'min',
    'GDP': 'mean',
    'year': 'max',
    'annual_earning_ft': 'mean',
    'unemployment_rate': 'mean',
    'youth_unemployment_rate': 'mean',
    'inflation_rate': 'mean',
    'gov_debt': 'mean',
    'tax_revenue': 'mean',
    'department_CO': lambda x: x.mode()[0],
    'department_CPS': lambda x: x.mode()[0],
    'department_DBT': lambda x: x.mode()[0],
    'department_DCMS': lambda x: x.mode()[0],
    'department_DEFRA': lambda x: x.mode()[0],
    'department_DEFRA & DFT': lambda x: x.mode()[0],
    'department_DESNZ': lambda x: x.mode()[0],
    'department_DFE': lambda x: x.mode()[0],
    'department_DFID': lambda x: x.mode()[0],
    'department_DFT': lambda x: x.mode()[0],
    'department_DHSC': lambda x: x.mode()[0],
    'department_DLUHC': lambda x: x.mode()[0],
    'department_DSIT': lambda x: x.mode()[0],
    'department_DWP': lambda x: x.mode()[0],
    'department_FCDO': lambda x: x.mode()[0],
    'department_HMLR': lambda x: x.mode()[0],
    'department_HMRC': lambda x: x.mode()[0],
    'department_HMT': lambda x: x.mode()[0],
    'department_HO': lambda x: x.mode()[0],
    'department_MOD': lambda x: x.mode()[0],
    'department_MOJ': lambda x: x.mode()[0],
    'department_NCA': lambda x: x.mode()[0],
    'department_NS&I': lambda x: x.mode()[0],
    'department_ONS': lambda x: x.mode()[0],
    'department_VOA': lambda x: x.mode()[0],
    'colour_rating_amber/green': lambda x: x.mode()[0],
    'colour_rating_amber/red': lambda x: x.mode()[0],
    'colour_rating_green': lambda x: x.mode()[0],
    'colour_rating_red': lambda x: x.mode()[0],
    'colour_rating_reset': lambda x: x.mode()[0],
    'topic': 'mean'
}

for col in colour_rating_columns:
    agg_dict[col] = lambda x, col=col: latest_value(x, col)

for col in department_columns:
    agg_dict[col] = lambda x, col=col: latest_value(x, col)

df = df.groupby('project_name').agg(agg_dict)


Error in Group department_CO for column department_CO: 'year'
Error in Group department_CO for column department_CO: 'year'
Error in Group department_CO for column department_CO: 'year'
Error in Group department_CO for column department_CO: 'year'
Error in Group department_CO for column department_CO: 'year'
Error in Group department_CO for column department_CO: 'year'
Error in Group department_CO for column department_CO: 'year'
Error in Group department_CO for column department_CO: 'year'
Error in Group department_CO for column department_CO: 'year'
Error in Group department_CO for column department_CO: 'year'
Error in Group department_CO for column department_CO: 'year'
Error in Group department_CO for column department_CO: 'year'
Error in Group department_CO for column department_CO: 'year'
Error in Group department_CO for column department_CO: 'year'
Error in Group department_CO for column department_CO: 'year'
Error in Group department_CO for column department_CO: 'year'
Error in

### Dropping & further modeling when not grouped

In [ ]:
df[df.select_dtypes(['object']).columns] = df.select_dtypes(['object']).astype(bool)

In [ ]:
df = df.drop(columns=['project_name', 'description_aims', 'rating_comment', 'start_date', 'end_date', 'schedule_comment', 'variance_comment', 'budget_comment', 'year', 'yearly_budget', 'yearly_forecast', 'report_category', 'project_number', 'total_benefits', 'benefits_comment', 'end_year', 'youth_unemployment_rate', 'population', 'revenue_excl_grants', 'grants_and_other_revenue', 'nlp', 'topic_distribution', 'topic_words'])

In [ ]:
df.columns.tolist()

### Dropping & further modeling when grouped

In [ ]:
df.columns.to_list()

In [4]:
df = df.drop(columns=['end_year','start_year','GDP','year','annual_earning_ft','unemployment_rate','youth_unemployment_rate','inflation_rate','gov_debt','tax_revenue','colour_rating_amber/green','colour_rating_amber/red','colour_rating_green','colour_rating_red','colour_rating_reset','topic'])

In [ ]:
df.columns.to_list()

In [5]:
df[df.select_dtypes(['object']).columns] = df.select_dtypes(['object']).astype(bool)

## Modeling

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

df_lr = df.dropna()

X = df_lr.drop(columns=['wlc_baseline_incl_NCG'])
y = df_lr['wlc_baseline_incl_NCG']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_})
print(coefficients)


In [6]:
from sklearn.preprocessing import RobustScaler

numerical_features = df.select_dtypes(include=['float64', 'int64']).columns

scaler = RobustScaler()

df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [9]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error

X = df.drop(columns=['wlc_baseline_incl_NCG'])
y = df['wlc_baseline_incl_NCG']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_reg = XGBRegressor(objective='reg:squarederror', random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

grid_search = GridSearchCV(estimator=xgb_reg, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f'Best Params: {best_params}')
print(f'Best negative MSE (Training): {best_score}')

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (Test): {mse}')


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Params: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Best negative MSE (Training): -1.0623074353811244
Mean Squared Error (Test): 0.8506409420204977


In [10]:
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 0.9223019798420133


In [11]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from bayes_opt import BayesianOptimization
from hyperopt import hp, fmin, tpe

In [12]:
from sklearn.datasets import make_regression

X = df[['year_duration'] + [col for col in df.columns if 'department_' in col]]
y = df['wlc_baseline_incl_NCG']


# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function to minimize
def xgb_eval(max_depth, learning_rate, subsample, colsample_bytree):
    params = {
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'n_estimators': 100,
        'objective': 'reg:squarederror',
        'random_state': 42,
    }

    model = XGBRegressor(**params)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    return score

# Define the search space
pbounds = {
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.3),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
}

# Perform Bayesian optimization
optimizer = BayesianOptimization(f=xgb_eval, pbounds=pbounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=25)

best_params = optimizer.max['params']

# Print the best hyperparameters and score
print(f"Best hyperparameters: {optimizer.max['params']}")
print(f"Best score: {optimizer.max['target']:.4f}")

|   iter    |  target   | colsam... | learni... | max_depth | subsample |
-------------------------------------------------------------------------
| 1         | -0.4512   | 0.6873    | 0.2857    | 8.124     | 0.7993    |
| 2         | 0.02107   | 0.578     | 0.05524   | 3.407     | 0.9331    |
| 3         | -0.1501   | 0.8006    | 0.2153    | 3.144     | 0.985     |
| 4         | -0.1033   | 0.9162    | 0.07158   | 4.273     | 0.5917    |
| 5         | -0.3418   | 0.6521    | 0.1622    | 6.024     | 0.6456    |
| 6         | 0.03898   | 0.5664    | 0.03288   | 3.35      | 0.9247    |
| 7         | 0.02965   | 0.5       | 0.01      | 3.343     | 0.5       |
| 8         | -0.006997 | 0.5       | 0.01      | 10.0      | 0.5       |
| 9         | -0.4269   | 1.0       | 0.3       | 10.0      | 1.0       |
| 10        | -0.004065 | 0.5       | 0.01      | 9.368     | 0.5       |
| 11        | -0.2106   | 0.5       | 0.3       | 4.756     | 1.0       |
| 12        | 0.0415    | 1.0       | 

In [13]:
from xgboost import XGBRegressor

# Round best_params 'max_depth' result by saving as integer
best_params['max_depth'] = int(best_params['max_depth'])

# Initialising model with best parameters
best_model = XGBRegressor(
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    n_estimators=100,
    objective='reg:squarederror',
    random_state=42
)

# Train the model with best parameters
best_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [14]:
y_pred = best_model.predict(X_test)
y_true = y_test
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (Test): {mse}')
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (Test): {rmse}')
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mape = mean_absolute_percentage_error(y_true, y_pred)
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

Mean Squared Error (Test): 0.8505931608439128
Root Mean Squared Error (Test): 0.9222760762612856
Mean Absolute Percentage Error (MAPE): 442.78%


In [15]:
import pickle

with open('best_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

In [16]:
with open("best_model.pkl", "rb") as file:
    loaded_model = pickle.load(file)

In [17]:
departments = ['department_CO',
 'department_CPS',
 'department_DBT',
 'department_DCMS',
 'department_DEFRA',
 'department_DEFRA & DFT',
 'department_DESNZ',
 'department_DFE',
 'department_DFID',
 'department_DFT',
 'department_DHSC',
 'department_DLUHC',
 'department_DSIT',
 'department_DWP',
 'department_FCDO',
 'department_HMLR',
 'department_HMRC',
 'department_HMT',
 'department_HO',
 'department_MOD',
 'department_MOJ',
 'department_NCA',
 'department_NS&I',
 'department_ONS',
 'department_VOA']

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin

# wrapper class for model
class ModelWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, model, departments):
        self.model = model
        self.departments = departments
    
    def transform_input(self, input_department, input_years):
        # dummies for departments
        department_dummy = pd.get_dummies([input_department], columns=self.departments)
        
        # missing dummies
        for department in self.departments:
            if department not in department_dummy.columns:
                department_dummy[department] = 0
        
        # ensure that deoartment dummies are in the correct order
        department_dummy = department_dummy[self.departments]

        # add years feature
        feature_array = np.concatenate([department_dummy.values.flatten(), [input_year]])

        return feature_array.reshape(1, -1)
    
    def predict(self, input_department, input_year):
        transformed_input = self.transform_input(input_department, input_year)
    
        return self.model.predict(transformed_input)

In [20]:
# initialising wrapper
model_wrapper = ModelWrapper(best_model, departments)
